In [9]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import pickle

# Функция для предобработки изображения
def preprocess_image(image_path, width, height):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        # Сглаживание изображения
        img = cv2.GaussianBlur(img, (5, 5), 0)
        # Адаптивное пороговое значение
        img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        # Изменение размера изображения
        img = cv2.resize(img, (width, height))
        # Нормализация изображения
        img = img.astype('float32') / 255.0
    return img

# Функция для загрузки модели
def load_trained_model(model_path):
    model = load_model(model_path)
    model.summary()  # Вывод структуры модели для определения ожидаемых входов
    return model

# Функция для получения предсказаний модели
def predict_sequence(model, image, max_seq_len, second_input_shape):
    # Создание второго входа с нужной формой
    second_input = np.zeros((1, second_input_shape[1]))  # Изменить здесь на правильную форму

    # Предсказание с двумя входами
    pred = model.predict([np.expand_dims(image, axis=0), second_input])
    
    pred_classes = np.argmax(pred, axis=-1)
    pred_classes_padded = pad_sequences(pred_classes, maxlen=max_seq_len, padding='post')
    return pred_classes_padded.flatten()

def tokens_to_latex(tokens, tokenizer):
    # Обратное отображение токенов в слова
    index_to_word = {index: word for word, index in tokenizer.word_index.items()}
    
    # Формирование LaTeX кода из предсказанных токенов
    latex_code = ''.join([index_to_word[token] for token in tokens if token != 0 and token in index_to_word])
    
    return latex_code

# Путь к обученной модели, токенайзеру и тестовому изображению
model_path = './best_model.keras'
tokenizer_path = './tokenizer.pkl'
image_path = './data_2019/3_images_test/ISICal19_1201_em_751.png'

# Загрузка модели
model = load_trained_model(model_path)

# Загрузка токенайзера
with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)

# Параметры изображения и максимальная длина последовательности
desired_width = 256
desired_height = 256
max_seq_len = 100  # максимальная длина последовательности, изменить при необходимости

# Загрузка и предобработка изображения
image = preprocess_image(image_path, desired_width, desired_height)

# Определение формы второго входа
second_input_shape = model.input[1].shape

# Получение предсказаний
pred_tokens = predict_sequence(model, image, max_seq_len, second_input_shape)

# Преобразование предсказанных токенов в LaTeX код
latex_code = tokens_to_latex(pred_tokens, tokenizer)

print(latex_code)



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ img_input           │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 256,  │        320 │ img_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │     18,496 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d_4[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 131072)    │          0 │ max_pooling2d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │ 33,554,688 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ seq_input           │ (None, 206)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 206, 256)  │          0 │ dropout_1[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 206, 256)  │     72,704 │ seq_input[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 206, 512)  │          0 │ repeat_vector_1[… │
│ (Concatenate)       │                   │            │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 206, 512)  │  1,574,912 │ concatenate[0][0

 Total params: 111,048,534 (423.62 MB)

 Trainable params: 37,016,028 (141.20 MB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 74,032,058 (282.41 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

